# Big Data

STARTED

1. Spark

https://courses.edx.org/courses/course-v1:BerkeleyX+CS105x+1T2016/course/

2. tips is run OOM oout of memory (very good tips)

https://courses.edx.org/courses/course-v1:BerkeleyX+CS105x+1T2016/courseware/d1f293d0cb53466dbb5c0cd81f55b45b/fe9a95cc542d4c30b855e632663c4797/8?activate_block_id=block-v1%3ABerkeleyX%2BCS105x%2B1T2016%2Btype%40vertical%2Bblock%4083ff2d3b4e93489b9b7b4861811e0872



# Spark

## Install Sparks, pyspark n findspark

1. needs docker
2. needs edx course docker image


```sh

# Install images
docker pull ucsddse230/cse255-dse230

# Run container
docker run --name ds_pragmatic -it -p 8890:8888 -v /media/leandroohf/sdb1/leandro/ds_pragmatic_programming:/home/ucsddse230/ ucsddse230/cse255-dse230 /bin/bash

# Run jupyter inside container
jupyter notebook
```

1. http://localhost:8889/tree
2. Copy and paste token to login in the notebook

In [ ]:
!ls data/Weather/

## Load modules

In [1]:
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.types import Row, StructField, StructType, StringType, IntegerType
%pylab inline

#start the SparkContext
import findspark
findspark.init()

from pyspark import SparkContext
sc = SparkContext(master="local[4]")

Populating the interactive namespace from numpy and matplotlib


## Spark: Operations on (key,val) RDDs

**Key/value pairs**

* A python dictionary is a collection of *key/value* pairs.
* The **key** is used to find a set of pairs with the particular key.
* The **value** can be anything.
* Spark has a set of special operations for *(key,value)* RDDs.

* For more, see the spark [RDD programming guide]( http://spark.apache.org/docs/latest/rdd-programming-guide.html)

### Creating RDDS



#### Creating from txt files

In [ ]:
!head data/Moby-Dick.txt

text_file = sc.textFile('data/Moby-Dick.txt')
type(text_file)

#### Creating (key,value) RDDS

**Method 1:** `parallelize` a list of pairs.

In [ ]:
pair_rdd = sc.parallelize([(1,2), (3,4)])
print(pair_rdd.collect())

**Method 2:** `map` a function that maps elements to key/value pairs.

In [ ]:
regular_rdd = sc.parallelize([1, 2, 3, 4, 2, 5, 6])
pair_rdd = regular_rdd.map( lambda x: (x, x*x) )
print(pair_rdd.collect())

#### Basic operations

**Properties of reduce operations**

* Reduce operations **must not depend on the order**
  * Order of operands should not matter
  * Order of application of reduce operator should not matter

* Multiplication and summation are good:

```
                1 + 3 + 5 + 2                      5 + 3 + 1 + 2 
```

 * Division and subtraction are bad:

```
                    1 - 3 - 5 - 2                      1 - 3 - 5 - 2
```


In [ ]:
A=sc.parallelize(range(3))

L=A.collect()
print(type(L))
print(L)

print()
print(A.map(lambda x: x*x).collect())

print()
print(A.reduce(lambda x,y:x+y))

# Does not work because depends of the order of execution wich you do not have control
# Will return different when running in different times or machines
print(A.reduce(lambda x,y:x-y))


#### Transformations on (key,value) rdds. ** RDD $\to$ RDD.
  * **Examples** map, filter, sample, [More](http://spark.apache.org/docs/latest/rdd-programming-guide.html#transformations)
  * **No** communication needed.

**Shuffles:** RDD $\to$ RDD, **shuffle** needed

**Shuffles are costly transfromations**

* **Examples:** sort, distinct, repartition, sortByKey, reduceByKey, join [More](http://spark.apache.org/docs/latest/rdd-programming-guide.html#shuffle-operations)
  * **A LOT** of communication might be needed.


##### reduceByKey(func)
Apply the reduce function on the values with the same key. 

In [ ]:
rdd = sc.parallelize([(1,2), (2,4), (2,6)])
print("Original RDD :", rdd.collect())
print("After transformation : ", rdd.reduceByKey(lambda a,b: a+b).collect())

Note that although it is similar to the reduce function, it is implemented as a transformation and not as an action because the dataset can have very large number of keys. So, it does not return values to the driver program. Instead, it returns a new RDD. 

##### sortByKey()  and sortBy(): 
Sort RDD by keys in ascending order. 


In [ ]:
rdd = sc.parallelize([(2,2), (1,4), (3,6)])
print("Original RDD :", rdd.collect())
print("After transformation : ", rdd.sortByKey().collect())

# Using sortBy
print("After transformation : ", rdd.sortBy(lambda x: x[1],ascending=False).collect())



**Note:** The output of sortByKey() is an RDD. This means that  RDDs do have a meaningful order, which extends between partitions.

##### mapValues(func):
Apply func to each value of RDD without changing the key. 

In [ ]:
rdd = sc.parallelize([(1,2), (2,4), (2,6)])
print("Original RDD :", rdd.collect())
print("After transformation : ", rdd.mapValues(lambda x: x*2).collect())

##### groupByKey(): 
Returns a new RDD of `(key,<iterator>)` pairs where the iterator iterates over the values associated with the key.

[Iterators](http://anandology.com/python-practice-book/iterators.html) are python objects that generate a sequence of values. Writing a loop over `n` elements as 
```python
for i in range(n):
    ##do something
```
is inefficient because it first allocates a list of `n` elements and then iterates over it.
Using the iterator `xrange(n)` achieves the same result without materializing the list. Instead, elements are generated on the fly.

To materialize the list of values returned by an iterator we will use the list comprehension command:
```python
[a for a in <iterator>]
```

In [ ]:
rdd = sc.parallelize([(1,2), (2,4), (2,6)])
print("Original RDD :", rdd.collect())
print("After transformation : ", rdd.groupByKey().mapValues(lambda x:[a for a in x]).collect())

In [ ]:
print("Original RDD :", rdd.collect())

# This sytanx is not working
#print(rdd.groupByKey().map(lambda k, _iter: (k,[x for x in _iter ])).collect())
print(rdd.groupByKey().map(lambda elem: (elem[0],[x for x in elem[1] ])).collect())


In [ ]:
# LHOF Notes
rdd = sc.parallelize([(1,2), (2,4), (2,6), (2,7)])
print("Original RDD :", rdd.collect())
# python 3 xrange = range and xrange is not defined anymore. This range is effcient
print("len(iterator):  ", len(range(5)))
print("After transformation : ", rdd.groupByKey().mapValues(lambda x: len(x)).collect())

#####  flatMapValues(func): 

Similar to `flatMap()`: creates a separate key/value pair for each element of the list generated by the map operation.

`func` is a function that takes as input a single value and returns an iterator that generates a sequence of values.
The application of flatMapValues operates on a key/value RDD. It applies `func` to each value, and gets an list (generated by the iterator) of values. It then combines each of the values with the original key to produce a list of key-value pairs. These lists are concatenated as in `flatMap`

In [ ]:
rdd = sc.parallelize([(1,2), (2,4), (2,6)])
print("Original RDD :", rdd.collect())

# LHOF Notes
x = 3
print('list: ', list(range(x,x+2)))

# the lambda function generates for each number i, an iterator that produces i,i+1
print("After transformation : ", rdd.flatMapValues(lambda x: list(range(x,x+2))).collect())

##### (Advanced)  combineByKey(createCombiner, mergeValue, mergeCombiner): 
Combine values with the same key using a different result type.

This is the most general of the per-key aggregation functions. Most of the other per-key combiners are implemented using it. 

The elements of the original RDD are considered here *values*

Values are converted into *combiners* which we will refer to here as "accumulators". An example of such a mapping is the mapping of the value *word* to the accumulator (*word*,1) that is done in WordCount.

accumulators are then combined with values and the other combiner to generate a result for each key.

For example, we can use it to calculate per-activity average durations as follows. Consider an RDD of key/value pairs where keys correspond to different activities and values correspond to duration.


In [ ]:
rdd = sc.parallelize([("Sleep", 7), ("Work",5), ("Play", 3), 
                      ("Sleep", 6), ("Work",4), ("Play", 4),
                      ("Sleep", 8), ("Work",5), ("Play", 5),("Sleep", 2)])

sum_counts = rdd.combineByKey(
    (lambda x: (x, 1)), # createCombiner maps each value into a  combiner (or accumulator)
    (lambda acc, value: (acc[0]+value, acc[1]+1)),
#mergeValue defines how to merge a accumulator with a value (saves on mapping each value to an accumulator first)
    (lambda acc1, acc2: (acc1[0]+acc2[0], acc1[1]+acc2[1])) # combine accumulators
)


print(rdd.collect())
print(sum_counts.collect())
duration_means_by_activity = sum_counts.mapValues(lambda value:
                                                  value[0]*1.0/value[1]) \
                                            .collect()
print(duration_means_by_activity)

To understand combineByKey(), it’s useful to think of how it handles each element it processes. As combineByKey() traverses through the elements in a partition, each element either has a key it hasn’t seen before or has the same key as a previous element.

If it’s a new key, createCombiner() is called to create the initial value for the accumulator on that key. In the above example, the accumulator is a tuple initialized as (x, 1) where x is a value in original RDD. Note that createCombiner() is called only when a key is seen for the first time in **each partition.**

If it is a key we have seen before while processing that partition, it will instead use the provided function, mergeValue(), with the current value for the accumulator for that key and the new value.

Since each partition is processed independently, we can have multiple accumulators for the same key. When we are merging the results from each partition, if two or more partitions have an accumulator for the same key, we merge the accumulators using the user-supplied mergeCombiners() function. In the above example, we are just adding the 2 accumulators element-wise.

### Transformations on two (key-value) RDDs

In [ ]:
rdd1 = sc.parallelize([(1,2),(2,1),(2,2)])
rdd2 = sc.parallelize([(2,5),(3,1)])
print('rdd1=',rdd1.collect())
print('rdd2=',rdd2.collect())

In [ ]:
# LHOF Notes

rdd1 = sc.parallelize([(1,2), (2,1), (2,2)])
rdd2 = sc.parallelize([(2,5), (3,1)])

print('rdd1: ', rdd1.collect())
print('rdd2: ', rdd2.collect())
print('subtractByKey: ', rdd1.subtractByKey(rdd2).collect())

print()
# Pay attention. This is a set operation
x = sc.parallelize([("a", 1), ("b", 4), ("b", 5), ("a", 3)])
y = sc.parallelize([("a", 3), ("c", None)])

print('x: ', x.collect())
print('y: ', y.collect())
print('subtract: ', sorted(x.subtract(y).collect()))

#### 1. subtractByKey: 
Remove from RDD1 all elements whose key is present in RDD2.

In [ ]:
print('rdd1=',rdd1.collect())
print('rdd2=',rdd2.collect())
print("Result:", rdd1.subtractByKey(rdd2).collect())

#### 2. join: 
* A fundamental operation in relational databases.
* assumes two tables have a **key** column in common. 
* merges rows with the same key.

Suppose we have two `(key,value)` datasets 

|**dataset 1**|                                     |..........| **dataset 2** | 	       	     |
|-------------|-------------------------------------|   |-------------|-----------------|
| **key=name**   |   **(gender,occupation,age)**    |   |  **key=name**   |   **hair color**    |
| John   |  (male,cook,21)                          |   | Jill   |  blond |
| Jill   |  (female,programmer,19)                  |   | Grace  |  brown |         
| John   |  (male, kid, 2)                          |   | John   |  black |
| Kate   |  (female, wrestler, 54)                  |

When `Join` is called on datasets of type `(Key, V)` and `(Key, W)`, it  returns a dataset of `(Key, (V, W))` pairs with all pairs of elements for each key. Joining the 2 datasets above yields:

|   key = name | (gender,occupation,age),haircolor |
|--------------|-----------------------------------|
| John         | ((male,cook,21),black)             |
| John         | ((male, kid, 2),black)             |
| Jill         | ((female,programmer,19),blond)     |

In [ ]:
print('rdd1=',rdd1.collect())
print('rdd2=',rdd2.collect())
print("Result:", rdd1.join(rdd2).collect())

####  Variants of join.

There are four variants of `join` which differ in how they treat keys that appear in one dataset but not the other.
* `join` is an *inner* join which means that keys that appear only in one dataset are eliminated.
* `leftOuterJoin` keeps all keys from the left dataset even if they don't appear in the right dataset. The result of leftOuterJoin in our example will contain the keys `John, Jill, Kate`
* `rightOuterJoin` keeps all keys from the right dataset even if they don't appear in the left dataset. The result of leftOuterJoin in our example will contain the keys `Jill, Grace, John`
* `FullOuterJoin` keeps all keys from both datasets. The result of leftOuterJoin in our example will contain the keys `Jill, Grace, John, Kate`

In outer joins, if the element appears only in one dataset, the element in `(K,(V,W))` that does not appear in the dataset is represented bye `None`

##### 3. rightOuterJoin: 
Perform a right join between two RDDs. Every key in the right/second RDD will be present at least once.

In [ ]:
print('rdd1=',rdd1.collect())
print('rdd2=',rdd2.collect())
print("Result:", rdd1.rightOuterJoin(rdd2).collect())

##### 4. leftOuterJoin: Perform a left join between two RDDs. Every key in the left RDD will be present at least once.

In [ ]:
print('rdd1=',rdd1.collect())
print('rdd2=',rdd2.collect())
print("Result:", rdd1.leftOuterJoin(rdd2).collect())

### Actions on (key,val) RDDs. RDD $\to$ Python-object in head node.

  * **Examples:** reduce, collect, count, take, [More](http://spark.apache.org/docs/latest/rdd-programming-guide.html#actions)
  * **Some** communication needed.

In [ ]:
rdd = sc.parallelize([(1,2), (2,4), (2,6)])

#### 1. countByKey(): 
Count the number of elements for each key. Returns a dictionary for easy access to keys.

In [ ]:
print("RDD: ", rdd.collect())
result = rdd.countByKey()
print("Result:", result)

#### 2. collectAsMap(): 
Collect the result as a dictionary to provide easy lookup.

In [ ]:
# LHOF: Use only you make sure there is NO duplicated key. Because you will not know the end results otherwise 
# and you might get different results in different runs

print("RDD: ", rdd.collect())
result = rdd.collectAsMap()
print("Result:", result)

#### 3. lookup(key): 
Return all values associated with the provided key.

In [ ]:
print("RDD: ", rdd.collect())
result = rdd.lookup(2 )
print("Result:", result)

### Word counts (Hello word)


In [ ]:
# Count the number of occurency of each word

words =     text_file.flatMap(lambda line: line.split(" "))
not_empty = words.filter(lambda x: x!='') 
key_values= not_empty.map(lambda word: (word, 1)) 
counts=     key_values.reduceByKey(lambda a, b: a + b)

In [ ]:
# Get thw top 5 most frequent word

reverse_counts=counts.map(lambda x:(x[1],x[0]))   # reverse order of word and count
sorted_counts=reverse_counts.sortByKey(ascending=False)

print(sorted_counts.take(5))

## Dataframes

Dataframe are just RDDs with shema (metinfo). Because of the restrictions imposed by the schema, they can be more optmized


sheet cheat:
https://s3.amazonaws.com/assets.datacamp.com/blog_assets/PySpark_SQL_Cheat_Sheet_Python.pdf

### Create Dataframes

In [2]:
# from pyspark.sql import Row
# #from pyspark import sqlContext

sqlContext = SQLContext(sc)

rdd = sc.parallelize([Row(name=u"John",  age=19),
                      Row(name=u"Smith", age=23),
                      Row(name=u"Sarah", age=18)])

rdd.collect()
# Output:
# [Row(name=u"John",  age=19),
#  Row(name=u"Smith", age=23),
#  Row(name=u"Sarah", age=18)]

df = sqlContext.createDataFrame(rdd)
df.printSchema()


root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



In [ ]:
#from pyspark.sql import StructType, StructField, StringType, IntegerType

rdd2 = sc.parallelize([("John",19),("Smith",23),("Sarah",18)])


schema = StructType([StructField("person_name", StringType(),False),
                     StructField("person_age",  IntegerType(),False)
])

df2 = sqlContext.createDataFrame(rdd2,schema)
df.printSchema()


### Read and write from disk


* parquet files (it is folder) are very popular e efficeent for IO in disk
* parqet can be query directly from the disk


In [ ]:
parquet_file = 'data/users.parquet'
print(parquet_file)

df = sqlContext.read.load(parquet_file)

df.show()

In [ ]:
parquet_file = 'data/users.parquet'
print(parquet_file)

df = sqlContext.read.load(parquet_file)

df.show()

In [ ]:
!rm -rv data/df.parquet

In [ ]:
# This throw an error if file already exit
df.write.save("df.parquet")
!ls 
!echo
!ls df.parquet

#### Query parquet file

In [3]:
# In this way because the parquet file was not distributed in the hadoop cluster you do not have parallelism
parquet_file = "data/Weather/NY.parquet"

query="""
SELECT station,measurement,year 
FROM parquet.`%s` 
WHERE measurement=\"SNOW\" 
"""%(parquet_file)

print(query)

df2 = sqlContext.sql(query)
print(df2.count(),df2.columns)

df2.show(5)


SELECT station,measurement,year 
FROM parquet.`data/Weather/NY.parquet` 
WHERE measurement="SNOW" 

15629 ['station', 'measurement', 'year']
+-----------+-----------+----+
|    station|measurement|year|
+-----------+-----------+----+
|USC00308600|       SNOW|1932|
|USC00308600|       SNOW|1956|
|USC00308600|       SNOW|1957|
|USC00308600|       SNOW|1958|
|USC00308600|       SNOW|1959|
+-----------+-----------+----+
only showing top 5 rows



### Data wrangler

#### Select columns

In [ ]:
df.select("name","favorite_color").show(5)

In [ ]:
!ls data/Weather

# In case the files was not un compressed
!tar -zxvf data/Weather/NY.tgz -C data/Weather

In [4]:
parquet_file = 'data/Weather/NY.parquet'
print(parquet_file)

df = sqlContext.read.load(parquet_file)

df.show(1)

data/Weather/NY.parquet
+-----------+-----------+----+--------------------+-----------------+--------------+------------------+-----------------+-----+-----------------+
|    Station|Measurement|Year|              Values|       dist_coast|      latitude|         longitude|        elevation|state|             name|
+-----------+-----------+----+--------------------+-----------------+--------------+------------------+-----------------+-----+-----------------+
|USW00094704|   PRCP_s20|1945|[00 00 00 00 00 0...|361.8320007324219|42.57080078125|-77.71330261230469|208.8000030517578|   NY|DANSVILLE MUNI AP|
+-----------+-----------+----+--------------------+-----------------+--------------+------------------+-----------------+-----+-----------------+
only showing top 1 row



In [5]:
df.describe().select('station','measurement').show()

+-----------+-----------+
|    station|measurement|
+-----------+-----------+
|     168398|     168398|
|       null|       null|
|       null|       null|
|USC00300015|       PRCP|
|USW00094794|   TOBS_s20|
+-----------+-----------+



### Aggregations

* count(),  
* avg(), max(), min()
* approx_count_distinct() which uses sampling to get an approximate count fast. **<=That is very cool. Based on inference**
 

In [6]:
df.groupby('measurement').agg({'year': 'min', 'station':'count'}).show(5)

+-----------+---------+--------------+
|measurement|min(year)|count(station)|
+-----------+---------+--------------+
|   TMIN_s20|     1873|         13442|
|       TMIN|     1873|         13442|
|   SNOW_s20|     1884|         15629|
|       TOBS|     1876|         10956|
|   SNWD_s20|     1888|         14617|
+-----------+---------+--------------+
only showing top 5 rows



In [7]:
print("True value: {}".format(df.select('station').distinct().count()))
df.agg({'station':'approx_count_distinct'}).show()

True value: 343
+------------------------------+
|approx_count_distinct(station)|
+------------------------------+
|                           339|
+------------------------------+



### Using SQL in a dataframe

In [ ]:
path = "data/people.json"

# Create a DataFrame from the file(s) pointed to by path
people = sqlContext.read.json(path)
#print('people is a',type(people))

print(people.count())

# The inferred schema can be visualized using the printSchema() method.
people.show()

In [ ]:
## Needs to register the dataframe first
people.registerTempTable("people")

teenagers = sqlContext.sql("SELECT name FROM people WHERE age >= 13 AND age <= 19")
for each in teenagers.collect():
    print(each[0])


### DataframeStatFunctions

Methods for statistics functionality. [documented here](http://takwatanabe.me/pyspark/generated/generated/pyspark.sql.DataFrameStatFunctions.html)

**Some of these functionso use inference to get aproximated estimations**

* **approxQuantile(col, probabilities, relativeError)**	Calculates the approximate quantiles of a numerical column of a DataFrame.
* **corr(col1, col2[, method])**	Calculates the correlation of two columns of a DataFrame as a double value.
* **cov(col1, col2)**	Calculate the sample covariance for the given columns, specified by their names, as a double value.
* **crosstab(col1, col2)**	Computes a pair-wise frequency table of the given columns.
* **freqItems(cols[, support])**	Finding frequent items for columns, possibly with false positives.
* **sampleBy(col, fractions[, seed])**	Returns a stratified sample without replacement based on the fraction given on each stratum.

In [8]:
# This make the quantiles
print([0.1*i for i in range(1,10)])


# That is very cool. Based on inference**
print()
print("Get the qiantiles")
print('with accuracy 0.1: ',df.approxQuantile('year', [0.1*i for i in range(1,10)], 0.1))
print('with accuracy 0.01: ',df.approxQuantile('year', [0.1*i for i in range(1,10)], 0.01))

[0.1, 0.2, 0.30000000000000004, 0.4, 0.5, 0.6000000000000001, 0.7000000000000001, 0.8, 0.9]

Get the qiantiles
with accuracy 0.1:  [1871.0, 1926.0, 1947.0, 1957.0, 1966.0, 1980.0, 1985.0, 1993.0, 2013.0]
with accuracy 0.01:  [1917.0, 1938.0, 1949.0, 1957.0, 1966.0, 1975.0, 1984.0, 1993.0, 2003.0]


### Dataframes UDF

    User define functions *UDF* are functions that can be ran against a
    dataframe and spark can optimize it

    Steps:
    1. define function that operates in the value field
    2. register the function
    3. use the function



In [12]:
from pyspark.sql.functions import udf

def packArray(a):
    """
    pack a numpy array into a bytearray that can be stored as a single 
    field in a spark DataFrame

    :param a: a numpy ndarray 
    :returns: a bytearray
    :rtype:

    """
    if type(a)!=np.ndarray:
        raise Exception("input to packArray should be numpy.ndarray. It is instead "+str(type(a)))
    return bytearray(a.tobytes())

def unpackArray(x,data_type=np.float16):
    """
    unpack a bytearray into a numpy.ndarray

    :param x: a bytearray
    :param data_type: The dtype of the array. This is important because if determines how many bytes go into each entry in the array.
    :returns: a numpy array
    :rtype: a numpy ndarray of dtype data_type.

    """
    return np.frombuffer(x,dtype=data_type)

def Count_nan(V):

    A=unpackArray(V,data_type=np.float16)

    return int(sum(np.isnan(A)))

#register
Count_nan_udf = udf(Count_nan, IntegerType())


In [13]:
df=df.withColumn("na_no", Count_nan_udf(df.Values))

In [17]:
df.select("Station","Year","Measurement","na_no").show(5)

+-----------+----+-----------+-----+
|    Station|Year|Measurement|na_no|
+-----------+----+-----------+-----+
|USW00094704|1945|   PRCP_s20|  204|
|USW00094704|1946|   PRCP_s20|    0|
|USW00094704|1947|   PRCP_s20|  143|
|USW00094704|1948|   PRCP_s20|    0|
|USW00094704|1949|   PRCP_s20|    0|
+-----------+----+-----------+-----+
only showing top 5 rows

